In [234]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices/sample_submission.csv
/kaggle/input/house-prices/train.csv
/kaggle/input/house-prices/test.csv


In [235]:
from sklearn.base import BaseEstimator, TransformerMixin

In [236]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.width', None)        
pd.set_option('display.expand_frame_repr', False)

In [237]:
df = pd.read_csv("/kaggle/input/house-prices/train.csv")
test_df = pd.read_csv("/kaggle/input/house-prices/test.csv")

In [238]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [239]:
from sklearn.model_selection import train_test_split

In [240]:
X_train, X_test = train_test_split(df, test_size=0.2, random_state=42)

y_train = X_train['SalePrice']
y_test = X_test['SalePrice']

X_train.drop(columns=["SalePrice"], inplace=True)
X_test.drop(columns=["SalePrice"], inplace=True)

X_train_saved = X_train.copy()

# Column Dropper, ModeImputer and Scaler classes for feature engineering

In [241]:
from sklearn.base import BaseEstimator, TransformerMixin
class ModeImputer(BaseEstimator, TransformerMixin):

    
    def fit(self, X, y=None):
        X_temp = X.copy()
        self.modes = X_temp.mode().iloc[0]
        return self
    
    def transform(self, X):
        X_transformed = X.copy()
        for col in X_transformed.columns:
            X_transformed[col] = X_transformed[col].fillna(self.modes[col])
        return X_transformed

In [242]:
class ColumnDropper(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_drop):
        self.columns_to_drop = columns_to_drop
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.drop(columns=self.columns_to_drop)

In [243]:
from sklearn.preprocessing import StandardScaler

class Scaler(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_scale):
        self.columns_to_scale = columns_to_scale
    
    def fit(self, X, y=None):
        X_cp = X.copy()
        self.scaler = StandardScaler()
        self.scaler.fit(X_cp[self.columns_to_scale])
        return self
    
    def transform(self, X):
        X_cp = X.copy()
        X_cp[self.columns_to_scale] = self.scaler.transform(X_cp[self.columns_to_scale])
        return X_cp

In [244]:
numeric_feats = X_train.select_dtypes(include=[np.number])

skewed_feats = numeric_feats.apply(lambda x: x.skew()).sort_values(ascending=False)

skewed_feats_cols = skewed_feats[skewed_feats > 1].index
# X_train[skewed_feats_cols] = np.log1p(X_train[skewed_feats_cols])

# X_train

In [245]:
from sklearn.preprocessing import StandardScaler

class SkewedFeatsHandler(BaseEstimator, TransformerMixin):
    def __init__(self, skewed_cols):
        self.skewed_cols = skewed_cols
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_cp = X.copy()
        X_cp[self.skewed_cols] = np.log1p(X_cp[self.skewed_cols])
        return X_cp

In [246]:
cat_cols = [col for col in X_train.columns if X_train[col].dtype == 'object']
cat_cols

['MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition']

In [247]:
cnts = X_train[cat_cols].nunique()
cnts

MSZoning          5
Street            2
Alley             2
LotShape          4
LandContour       4
Utilities         2
LotConfig         5
LandSlope         3
Neighborhood     25
Condition1        9
Condition2        8
BldgType          5
HouseStyle        8
RoofStyle         6
RoofMatl          7
Exterior1st      15
Exterior2nd      16
MasVnrType        3
ExterQual         4
ExterCond         5
Foundation        6
BsmtQual          4
BsmtCond          4
BsmtExposure      4
BsmtFinType1      6
BsmtFinType2      6
Heating           6
HeatingQC         5
CentralAir        2
Electrical        4
KitchenQual       4
Functional        7
FireplaceQu       5
GarageType        6
GarageFinish      3
GarageQual        5
GarageCond        5
PavedDrive        3
PoolQC            3
Fence             4
MiscFeature       4
SaleType          9
SaleCondition     6
dtype: int64

In [248]:
threshold = 3

target_encoded_cols = list(cnts[cnts > 3].index)
one_hot_cols = list(cnts[cnts <= 3].index)
target_encoded_cols

['MSZoning',
 'LotShape',
 'LandContour',
 'LotConfig',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageQual',
 'GarageCond',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition']

In [249]:
one_hot_cols

['Street',
 'Alley',
 'Utilities',
 'LandSlope',
 'MasVnrType',
 'CentralAir',
 'GarageFinish',
 'PavedDrive',
 'PoolQC']

In [250]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

class CategoricalPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, target_encoded_cols, one_hot_columns):
        self.target_encoded_cols = target_encoded_cols
        self.one_hot_columns = one_hot_columns 

    def fit(self, X, y):
        
        df_woe = X.copy()
        target_col = 'SalePrice'
        df_woe[target_col] = y

        woe_mappings = {}
        iv_values = {}

        self.encodings = {}
        global_mean = y.mean()
        
        for col in self.target_encoded_cols:
            # print(f"Processing {col}...")
            encoding_dict = df_woe.groupby(col)[target_col].mean().to_dict()
            # print(encoding_dict)
            
            encoding_dict['_default_'] = y.mean()
            
            self.encodings[col] = encoding_dict

        encoder = ColumnTransformer(
            transformers=[
                ('cat', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), self.one_hot_columns)
            ],
            remainder='passthrough',
            verbose_feature_names_out=False
        )

        encoder.fit(X)
        
        self.encoder = encoder

        return self


    def transform(self, X):
        X_transformed = X.copy()


        print("Preprocessing One Hot Columns")
        X_transformed_encoded = self.encoder.transform(X_transformed)
        encoded_feature_names = self.encoder.get_feature_names_out()
        X_transformed = pd.DataFrame(X_transformed_encoded, columns=encoded_feature_names)
        
       
        print("***")
        print("Preprocessing WOE Columns")
        for col in self.target_encoded_cols:
            X_transformed[col] = X_transformed[col].apply(
                lambda x: self.encodings[col].get(x, self.encodings[col]['_default_'])
            )
            # X_transformed.drop(columns=col, inplace=True)
            
        return X_transformed

# RFE

In [252]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression,Ridge

pipeline = Pipeline([
    ('ColumnDropper', ColumnDropper(["Id"])),
    ('NAHandling', ModeImputer()),
    ('SkewedFeatsScaler', SkewedFeatsHandler(skewed_feats_cols)),
    ('HandlingCategoricalValues', CategoricalPreprocessor(target_encoded_cols, one_hot_cols)),
    ('Scaler', Scaler(target_encoded_cols)),
    ('Model', LinearRegression())
])

In [253]:
# model = LinearRegression().fit(X_train, y_train)
pipeline.fit(X_train, y_train)

Preprocessing One Hot Columns
***
Preprocessing WOE Columns


Pipeline(steps=[('ColumnDropper', ColumnDropper(columns_to_drop=['Id'])),
                ('NAHandling', ModeImputer()),
                ('SkewedFeatsScaler',
                 SkewedFeatsHandler(skewed_cols=Index(['MiscVal', 'PoolArea', 'LotArea', '3SsnPorch', 'LowQualFinSF',
       'KitchenAbvGr', 'BsmtFinSF2', 'ScreenPorch', 'BsmtHalfBath',
       'EnclosedPorch', 'LotFrontage', 'OpenPorchSF', 'MasVnrArea',
       'BsmtFinSF1', 'Tota...
                                          'Condition1', 'Condition2',
                                          'BldgType', 'HouseStyle', 'RoofStyle',
                                          'RoofMatl', 'Exterior1st',
                                          'Exterior2nd', 'ExterQual',
                                          'ExterCond', 'Foundation', 'BsmtQual',
                                          'BsmtCond', 'BsmtExposure',
                                          'BsmtFinType1', 'BsmtFinType2',
                                          'Heating', 'HeatingQC', 'Electrical',
                                          'KitchenQual', 'Functional',
                                          'FireplaceQu', 'GarageType',
                                          'GarageQual', 'GarageCond', ...])),
                ('Model', LinearRegression())])

In [254]:
# y_train_pred = model.predict(X_train)
y_train_pred = pipeline.predict(X_train)

Preprocessing One Hot Columns
***
Preprocessing WOE Columns


In [255]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

def evaluate(y_true, y_pred, dataset="Set"):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = mse ** 0.5
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    print(f"{dataset} Performance:")
    print(f"MAE: {mae:.2f}")
    print(f"MAPE: {mape:.2f}")
    print(f"MSE: {mse:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"R² Score: {r2:.2f}\n")


evaluate(y_train, y_train_pred, "Train")

Train Performance:
MAE: 18193.95
MAPE: 0.11
MSE: 697795439.44
RMSE: 26415.82
R² Score: 0.88



In [256]:
from sklearn.model_selection import cross_val_score, KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)
rmse_scores = np.sqrt(-cross_val_score(pipeline, X_train, y_train, cv=kf, 
                                     scoring='neg_mean_squared_error'))

print("RMSE for each fold:", rmse_scores)
print(f"Average RMSE: {np.mean(rmse_scores):.4f}")

Preprocessing One Hot Columns
***
Preprocessing WOE Columns
Preprocessing One Hot Columns
***
Preprocessing WOE Columns
Preprocessing One Hot Columns
***
Preprocessing WOE Columns
Preprocessing One Hot Columns
***
Preprocessing WOE Columns
Preprocessing One Hot Columns
***
Preprocessing WOE Columns
Preprocessing One Hot Columns
***
Preprocessing WOE Columns


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2, 8] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Preprocessing One Hot Columns
***
Preprocessing WOE Columns
Preprocessing One Hot Columns
***
Preprocessing WOE Columns
Preprocessing One Hot Columns
***
Preprocessing WOE Columns
Preprocessing One Hot Columns
***
Preprocessing WOE Columns
RMSE for each fold: [30753.3917329  32710.74431348 39231.46458448 27848.55461158
 28538.78946334]
Average RMSE: 31816.5889


In [257]:
y_test_pred = pipeline.predict(X_test)
evaluate(y_test, y_test_pred)

Preprocessing One Hot Columns
***
Preprocessing WOE Columns
Set Performance:
MAE: 20779.41
MAPE: 0.12
MSE: 1099278881.38
RMSE: 33155.37
R² Score: 0.86



In [258]:
!pip install mlflow dagshub --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 50.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 74.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.0/684.0 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. Th

In [259]:
import mlflow
import dagshub
dagshub.init(repo_owner='Luka-Surmanidze', repo_name='MachineLearning', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=2f912ed1-6fcf-4eee-a731-2feb0cb33d05&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=64f22e9106a2505bd2ed47f5ee5d22b68828b00b89126bc7f4c314834e06b6bf




Output()

Accessing as Luka-Surmanidze

Initialized MLflow to track repo "Luka-Surmanidze/MachineLearning"

Repository Luka-Surmanidze/MachineLearning initialized!

In [260]:
experiment = "LiearRegression, second experiment for hw1"
run_name="Linear RegressionV2"

In [261]:
mlflow.set_experiment(experiment)

2025/04/11 19:10:11 INFO mlflow.tracking.fluent: Experiment with name 'LiearRegression, second experiment for hw1' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/7a2d046932c94ee0b3391e63b976655e', creation_time=1744398611270, experiment_id='2', last_update_time=1744398611270, lifecycle_stage='active', name='LiearRegression, second experiment for hw1', tags={}>

In [269]:

mlflow.end_run()
mlflow.start_run(run_name=run_name)

mlflow.log_param("model_type", "LinearRegression")
mlflow.log_param("random_state", 42)
mlflow.log_param("skewed_features_count", len(skewed_feats_cols))
mlflow.log_param("one_hot_threshold", 3)
mlflow.log_param("encoding_method", "OneHotEncoder/TargetEncoding")


pipeline = Pipeline([
    ('ColumnDropper', ColumnDropper(["Id"])),
    ('NAHandling', ModeImputer()),
    ('SkewedFeatsScaler', SkewedFeatsHandler(skewed_feats_cols)),
    ('HandlingCategoricalValues', CategoricalPreprocessor(target_encoded_cols, one_hot_cols)),
    ('Scaler', Scaler(target_encoded_cols)),
    ('Model', LinearRegression())
])

pipeline.fit(X_train, y_train)

y_train_pred = pipeline.predict(X_train)

y_test_pred = pipeline.predict(X_test)



data_sets = [[y_train, y_train_pred, "train"], [y_test, y_test_pred, "test"]]

kf = KFold(n_splits=5, shuffle=True, random_state=42)
rmse_scores = np.sqrt(-cross_val_score(pipeline, X_train, y_train, cv=kf, 
                                     scoring='neg_mean_squared_error'))

mlflow.log_metric("Average RMSE", float(np.mean(rmse_scores)))



for s in data_sets:
    y_true = s[0]
    y_pred = s[1]
    dataset = s[2]
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = mse ** 0.5
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    mlflow.log_metrics({
        f'{dataset}_mae': mae,
        f'{dataset}_mape': mape,
        f'{dataset}_mse': mse,
        f'{dataset}_rmse': rmse,
        f'{dataset}_r2': r2
    })
    print(f"{dataset} Performance:")
    print(f"MAE: {mae:.2f}")
    print(f"MAPE: {mape:.2f}")
    print(f"MSE: {mse:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"R² Score: {r2:.2f}\n")


mlflow.sklearn.log_model(model, "LinearRegressionV2")


mlflow.end_run()

🏃 View run Linear RegressionV2 at: https://dagshub.com/Luka-Surmanidze/MachineLearning.mlflow/#/experiments/2/runs/52935421d3704b088b2ac95e5bc6856a
🧪 View experiment at: https://dagshub.com/Luka-Surmanidze/MachineLearning.mlflow/#/experiments/2
Preprocessing One Hot Columns
***
Preprocessing WOE Columns
Preprocessing One Hot Columns
***
Preprocessing WOE Columns
Preprocessing One Hot Columns
***
Preprocessing WOE Columns
Preprocessing One Hot Columns
***
Preprocessing WOE Columns
Preprocessing One Hot Columns
***
Preprocessing WOE Columns
Preprocessing One Hot Columns
***
Preprocessing WOE Columns
Preprocessing One Hot Columns
***
Preprocessing WOE Columns
Preprocessing One Hot Columns
***
Preprocessing WOE Columns
Preprocessing One Hot Columns
***
Preprocessing WOE Columns
Preprocessing One Hot Columns
***
Preprocessing WOE Columns


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2, 8] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Preprocessing One Hot Columns
***
Preprocessing WOE Columns
Preprocessing One Hot Columns
***
Preprocessing WOE Columns
Preprocessing One Hot Columns
***
Preprocessing WOE Columns
train Performance:
MAE: 18193.95
MAPE: 0.11
MSE: 697795439.44
RMSE: 26415.82
R² Score: 0.88

test Performance:
MAE: 20779.41
MAPE: 0.12
MSE: 1099278881.38
RMSE: 33155.37
R² Score: 0.86



2025/04/11 19:16:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Linear RegressionV2 at: https://dagshub.com/Luka-Surmanidze/MachineLearning.mlflow/#/experiments/2/runs/c885fbff1be349088810ca2ef9c328ee
🧪 View experiment at: https://dagshub.com/Luka-Surmanidze/MachineLearning.mlflow/#/experiments/2


# Experiment 3

In [270]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression,Ridge

pipeline = Pipeline([
    ('ColumnDropper', ColumnDropper(["Id"])),
    ('NAHandling', ModeImputer()),
    ('SkewedFeatsScaler', SkewedFeatsHandler(skewed_feats_cols)),
    ('HandlingCategoricalValues', CategoricalPreprocessor(target_encoded_cols, one_hot_cols)),
    ('Scaler', Scaler(target_encoded_cols)),
    ('Model', Ridge(alpha=1.0))
])

In [279]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'Model__alpha': [0.01, 0.1, 1.0, 10.0, 100.0], 
    'RFE__n_features_to_select': [30, 40, 50, 60, 70]
}


grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1  
)


grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)
print("Best RMSE:", np.sqrt(-grid_search.best_score_))


best_model = grid_search.best_estimator_
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)


evaluate(y_train, y_train_pred, "Train")
evaluate(y_test, y_test_pred, "Test")

Preprocessing One Hot Columns
***
Preprocessing WOE Columns
Best parameters: {'Model__alpha': 100.0, 'RFE__n_features_to_select': 70}
Best RMSE: 33384.10808826466
Preprocessing One Hot Columns
***
Preprocessing WOE Columns
Preprocessing One Hot Columns
***
Preprocessing WOE Columns
Train Performance:
MAE: 18026.16
MAPE: 0.11
MSE: 719774801.47
RMSE: 26828.62
R² Score: 0.88

Test Performance:
MAE: 20560.77
MAPE: 0.12
MSE: 1125914901.95
RMSE: 33554.66
R² Score: 0.85



In [280]:
experiment = "Ridge, third experiment for hw1"
run_name="Ridge"

In [281]:
mlflow.set_experiment(experiment)

<Experiment: artifact_location='mlflow-artifacts:/ecc995cc2eb44f8c918535e7172b8663', creation_time=1744400097361, experiment_id='3', last_update_time=1744400097361, lifecycle_stage='active', name='Ridge, third experiment for hw1', tags={}>

In [282]:

mlflow.end_run()
mlflow.start_run(run_name=run_name)

mlflow.log_param("model_type", "LinearRegression")
mlflow.log_param("random_state", 42)
mlflow.log_param("skewed_features_count", len(skewed_feats_cols))
mlflow.log_param("one_hot_threshold", 3)
mlflow.log_param("encoding_method", "OneHotEncoder/TargetEncoding")


data_sets = [[y_train, y_train_pred, "train"], [y_test, y_test_pred, "test"]]

mlflow.log_metric("best_RMSE_score", float(np.sqrt(-grid_search.best_score_)))



for s in data_sets:
    y_true = s[0]
    y_pred = s[1]
    dataset = s[2]
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = mse ** 0.5
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    mlflow.log_metrics({
        f'{dataset}_mae': mae,
        f'{dataset}_mape': mape,
        f'{dataset}_mse': mse,
        f'{dataset}_rmse': rmse,
        f'{dataset}_r2': r2
    })
    print(f"{dataset} Performance:")
    print(f"MAE: {mae:.2f}")
    print(f"MAPE: {mape:.2f}")
    print(f"MSE: {mse:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"R² Score: {r2:.2f}\n")


mlflow.sklearn.log_model(best_model, "Ridge")


mlflow.end_run()

train Performance:
MAE: 18026.16
MAPE: 0.11
MSE: 719774801.47
RMSE: 26828.62
R² Score: 0.88

test Performance:
MAE: 20560.77
MAPE: 0.12
MSE: 1125914901.95
RMSE: 33554.66
R² Score: 0.85



2025/04/11 20:03:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Ridge with RFE and StandardScaler at: https://dagshub.com/Luka-Surmanidze/MachineLearning.mlflow/#/experiments/3/runs/394ac35b3c1a44c69b36cf0d99c8380c
🧪 View experiment at: https://dagshub.com/Luka-Surmanidze/MachineLearning.mlflow/#/experiments/3


# Ridge with StandardScaler and RFE

In [283]:
from sklearn.feature_selection import RFE
from sklearn.base import BaseEstimator, TransformerMixin

class RFETransformer(BaseEstimator, TransformerMixin):
    def __init__(self, estimator=None, n_features_to_select=None, step=1):
        self.estimator = estimator
        self.n_features_to_select = n_features_to_select
        self.step = step
        self.selected_features_ = None
        self.rfe = None
        
    def fit(self, X, y=None):
        self.rfe = RFE(
            estimator=self.estimator,
            n_features_to_select=self.n_features_to_select,
            step=self.step
        )
        
    
        self.rfe.fit(X, y)
        
   
        if hasattr(X, 'columns'):
            self.selected_features_ = X.columns[self.rfe.support_].tolist()
        
        return self
    
    def transform(self, X):
      
        X_transformed = X.copy()
        
        if hasattr(X, 'columns'):
            
            return X_transformed[self.selected_features_]
        else:
            
            return self.rfe.transform(X_transformed)

In [284]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression,Ridge

pipeline = Pipeline([
    ('ColumnDropper', ColumnDropper(["Id"])),
    ('NAHandling', ModeImputer()),
    ('SkewedFeatsScaler', SkewedFeatsHandler(skewed_feats_cols)),
    ('HandlingCategoricalValues', CategoricalPreprocessor(target_encoded_cols, one_hot_cols)),
    ('Scaler', StandardScaler()),  # Scale all features (or you can use your custom Scaler with all numeric columns)
    ('RFE', RFETransformer(estimator=Ridge(alpha=100.0), n_features_to_select=50, step=1)),
    ('Model', Ridge(alpha=1.0))
])

In [285]:
from sklearn.model_selection import GridSearchCV


param_grid = {
    'Model__alpha': [0.01, 0.1, 1.0, 10.0, 100.0],  # Test different alpha values
    'RFE__n_features_to_select': [30, 40, 50, 60, 70]
}


grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1  
)


grid_search.fit(X_train, y_train)


print("Best parameters:", grid_search.best_params_)
print("Best RMSE:", np.sqrt(-grid_search.best_score_))


best_model = grid_search.best_estimator_
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)


evaluate(y_train, y_train_pred, "Train")
evaluate(y_test, y_test_pred, "Test")

Preprocessing One Hot Columns
***
Preprocessing WOE Columns
Best parameters: {'Model__alpha': 100.0, 'RFE__n_features_to_select': 70}
Best RMSE: 33431.519454062305
Preprocessing One Hot Columns
***
Preprocessing WOE Columns
Preprocessing One Hot Columns
***
Preprocessing WOE Columns
Train Performance:
MAE: 18011.11
MAPE: 0.11
MSE: 719690120.30
RMSE: 26827.04
R² Score: 0.88

Test Performance:
MAE: 20536.76
MAPE: 0.12
MSE: 1118891092.20
RMSE: 33449.83
R² Score: 0.85



In [286]:
experiment = "Ridge, third experiment for hw1"
run_name="Ridge with RFE(model is Ridge(100)) and StandardScaler"
mlflow.set_experiment(experiment)

<Experiment: artifact_location='mlflow-artifacts:/ecc995cc2eb44f8c918535e7172b8663', creation_time=1744400097361, experiment_id='3', last_update_time=1744400097361, lifecycle_stage='active', name='Ridge, third experiment for hw1', tags={}>

In [287]:

mlflow.end_run()
mlflow.start_run(run_name=run_name)

mlflow.log_param("model_type", "LinearRegression")
mlflow.log_param("random_state", 42)
mlflow.log_param("skewed_features_count", len(skewed_feats_cols))
mlflow.log_param("one_hot_threshold", 3)
mlflow.log_param("encoding_method", "OneHotEncoder/TargetEncoding")


data_sets = [[y_train, y_train_pred, "train"], [y_test, y_test_pred, "test"]]

mlflow.log_metric("best_RMSE_score", float(np.sqrt(-grid_search.best_score_)))



for s in data_sets:
    y_true = s[0]
    y_pred = s[1]
    dataset = s[2]
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = mse ** 0.5
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    mlflow.log_metrics({
        f'{dataset}_mae': mae,
        f'{dataset}_mape': mape,
        f'{dataset}_mse': mse,
        f'{dataset}_rmse': rmse,
        f'{dataset}_r2': r2
    })
    print(f"{dataset} Performance:")
    print(f"MAE: {mae:.2f}")
    print(f"MAPE: {mape:.2f}")
    print(f"MSE: {mse:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"R² Score: {r2:.2f}\n")


mlflow.sklearn.log_model(best_model, "Ridge")


mlflow.end_run()

train Performance:
MAE: 18011.11
MAPE: 0.11
MSE: 719690120.30
RMSE: 26827.04
R² Score: 0.88

test Performance:
MAE: 20536.76
MAPE: 0.12
MSE: 1118891092.20
RMSE: 33449.83
R² Score: 0.85



2025/04/11 20:22:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Ridge with RFE(model is Ridge(100)) and StandardScaler at: https://dagshub.com/Luka-Surmanidze/MachineLearning.mlflow/#/experiments/3/runs/c083033f713240a98808391c3edaacfa
🧪 View experiment at: https://dagshub.com/Luka-Surmanidze/MachineLearning.mlflow/#/experiments/3
